In [26]:
%load_ext autoreload
%autoreload 2

from uniswap import Uniswap
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from BinanceClient import BinanceClient 
from CoinbaseClient import CoinbaseClient 
from web3 import Web3, HTTPProvider
import json
from Token import Token
from binance.exceptions import BinanceAPIException
from datetime import datetime
import time
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
address = Web3.toChecksumAddress("0x4975AE6f6991ab7D68AF65B8e184fcd6FC1b87cf".lower())
private_key = "87c1fc44deb9c7b1445c210c471ba0f9e364f02dfe862d5a46b2eb4ffc6c1621" 
version = 2         
provider = "https://mainnet.infura.io/v3/29613604d6144b348e4bf184cc80a40a"    
web3 = Web3(Web3.HTTPProvider(provider))
uniswap = Uniswap(address=address, private_key=private_key, version=version, provider=provider)
abi = '[{"constant":true,"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"addedValue","type":"uint256"}],"name":"increaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"mint","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"owner","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"subtractedValue","type":"uint256"}],"name":"decreaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"}]'
abi = json.loads(abi)

In [28]:
def filter_balance(x):
    if float(x["free"]) != 0 or float(x["locked"]) != 0:
        return True
    else:
        return False
    
binance_client = BinanceClient().client
all_tickers = binance_client.get_all_tickers()
account = binance_client.get_account()
balances = account["balances"]
balances = list(filter(filter_balance, balances))
balances

[{'asset': 'LDUSDT', 'free': '2014.96829564', 'locked': '0.00000000'},
 {'asset': 'DAI', 'free': '0.00000003', 'locked': '0.00000000'},
 {'asset': 'LUNA', 'free': '0.00000001', 'locked': '0.00000000'},
 {'asset': 'DYDX', 'free': '18.62714950', 'locked': '0.00000000'},
 {'asset': 'APE', 'free': '1.16315713', 'locked': '0.00000000'},
 {'asset': 'LUNC', 'free': '0.00288000', 'locked': '0.00000000'}]

In [36]:
coinbase_client = CoinbaseClient().client
coinbase_client
user = coinbase_client.get_current_user()


In [40]:
import requests

url = "https://api.exchange.coinbase.com/products/product_id/book?level=1"

headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"message":"NotFound"}


## Get Binance price

In [37]:
def binance_price(symbol, quantity):        
    depth = binance_client.get_order_book(symbol=symbol)        
    
    def weight_avg_order_prices(bid_or_ask, quantity):        
        avg = []    
        for order in bid_or_ask:
            order_price = float(order[0])
            order_qty = float(order[1])
            
            quantity -= order_qty
            if quantity >= 0:
                avg.append((order_price, order_qty))
            else:
                avg.append((order_price, order_qty+quantity))
                break
                        
        avg = sum([x[0]*x[1] for x in avg]) / sum(x[1] for x in avg)        
        return avg
               
    bid = weight_avg_order_prices(depth["bids"], quantity)
    ask = weight_avg_order_prices(depth["asks"], quantity)                           
    return bid, ask 

## Get Coinbase price

In [39]:
coinbase_client.get_buy_price(currency_pair = 'BTC-USD')
coinbase_client.get_orders()

NotFoundError: APIError(id=not_found): Not found

## 1. Search for arbitrage opportunity

In [32]:
known_pairs = [
    ("ETH", "USDT"),
    ("ETH", "DAI"),
    ("AAVE", "USDT"),
    ("UNI", "USDT"),
    ("BAT", "USDT"),
    ("SHIB", "USDT"),
    ("SAND", "USDT"),
    #("ACH", "USDT"),
]

In [33]:
tokenA = Token("ETH")
tokenB = Token("USDT")
binance_symbol = str(tokenA) + str(tokenB)

trade_size = 10000 # in tokenB

In [35]:
binance_symbol = str(tokenA) + str(tokenB)    
quantity = round(trade_size / binance_price(binance_symbol, 1)[0])
res = {}

print("Quantity = {}".format(quantity))
print()

binance_bid, binance_ask = binance_price(binance_symbol, quantity)
uni_price_bid = uniswap.get_price_input(tokenA.address,  tokenB.address,  quantity*10**tokenA.decimal) / (quantity*10**tokenB.decimal)
uni_price_ask = uniswap.get_price_output(tokenB.address, tokenA.address,  quantity*10**tokenA.decimal) / (quantity*10**tokenB.decimal)

assert(binance_ask >= binance_bid)
assert(uni_price_ask >= uni_price_bid)

# Case 1: buy tokenA from binance and convert tokenA to tokenB via uniqswap
profit_1 = (uni_price_bid - binance_ask) * quantity

print("Case 1: buy {} from binance and convert {} to {} via uniqswap".format(tokenA, tokenA, tokenB))
print("Binance buy for:  {:.3f}  | Uniswap sell for {:.3f}  | profit = {:.2f}".format(binance_ask, uni_price_bid, profit_1))
print()

# Case 2: convert tokenB to tokenA via uniswap and sell tokenA to binance
profit_2 = (binance_bid - uni_price_ask) * quantity
print("Case 2: convert DAI to ETH via uniswap and sell ETH to binance".format(tokenB, tokenA, tokenA))
print("Binance sell for: {:.3f}  | Uniswap buy for  {:.3f}  | profit = {:.2f}".format(binance_bid, uni_price_ask, profit_2))

Quantity = 8

Case 1: buy ETH from binance and convert ETH to USDT via uniqswap
Binance buy for:  1224.951  | Uniswap sell for 1215.565  | profit = -75.08

Case 2: convert DAI to ETH via uniswap and sell ETH to binance
Binance sell for: 1224.588  | Uniswap buy for  1223.805  | profit = 6.27


In [22]:
# res_arr = []
# res_arr

# while True:
#     res_arr = []    
#     for pairs in known_pairs:           
#         try:
#             tokenA = Token(pairs[0])
#             tokenB = Token(pairs[1])
#             binance_symbol = str(tokenA) + str(tokenB)    
#             quantity = round(trade_size / binance_price(binance_symbol, 1)[0])
#             res = {}

#             #print("Quantity = {}".format(quantity))
#             #print()

#             binance_bid, binance_ask = binance_price(binance_symbol, quantity)
#             uni_price_bid = uniswap.get_price_input(tokenA.address,  tokenB.address,  quantity*10**tokenA.decimal) / (quantity*10**tokenB.decimal)
#             uni_price_ask = uniswap.get_price_output(tokenB.address, tokenA.address,  quantity*10**tokenA.decimal) / (quantity*10**tokenB.decimal)

#             assert(binance_ask >= binance_bid)
#             assert(uni_price_ask >= uni_price_bid)

#             # Case 1: buy tokenA from binance and convert tokenA to tokenB via uniqswap
#             profit_1 = (uni_price_bid - binance_ask) * quantity

#             #print("Case 1: buy {} from binance and convert {} to {} via uniqswap".format(tokenA, tokenA, tokenB))
#             #print("Binance buy for:  {:.3f}  | Uniswap sell for {:.3f}  | profit = {:.2f}".format(binance_ask, uni_price_bid, profit_1))
#             #print()

#             # Case 2: convert tokenB to tokenA via uniswap and sell tokenA to binance
#             profit_2 = (binance_bid - uni_price_ask) * quantity
#             #print("Case 2: convert DAI to ETH via uniswap and sell ETH to binance".format(tokenB, tokenA, tokenA))
#             #print("Binance sell for: {:.3f}  | Uniswap buy for  {:.3f}  | profit = {:.2f}".format(binance_bid, uni_price_ask, profit_2))

#             res["tokenA"] = str(tokenA)
#             res["tokenB"] = str(tokenB)
#             res["binance_symbol"] = binance_symbol
#             res["quantity"] = quantity
#             res["trade_size"] = trade_size
#             res["binance_bid"] = binance_bid
#             res["binance_ask"] = binance_ask
#             res["uni_price_bid"] = uni_price_bid
#             res["uni_price_ask"] = uni_price_ask
#             res["profit_1"] = profit_1
#             res["profit_2"] = profit_2 
#             res["time"] = datetime.now()                                    
#             res_arr.append(res)                     
#             #print("{}: {}".format(res["time"], binance_symbol))                                    
#         except:
#             pass
        
#     df_temp = pd.DataFrame(res_arr)
#     output_path = "result.csv"
#     df_temp.to_csv(output_path, mode='a', header=not os.path.exists(output_path))        
#     time.sleep(30)        

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/6y/63prsvg51lgc4rgxgygg4xy80000gn/T/ipykernel_13766/3904268067.py", line 56, in <module>
    time.sleep(30)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1101, in get_records

TypeError: object of type 'NoneType' has no len()

## 2. Execute the arbitrage



## Trade Uniswap

In [ ]:
#qty_req_to_buy = uniswap.get_price_input(eth, dai, quantity)
uniswap.make_trade_output(dai, eth, quantity)

## Transfer from Binance to MetaMask wallet

In [ ]:
# Network = BSC or BUSD
try:
    result = client.withdraw(
        coin='ETH',
        address=address,
        amount=0.50000000,
        network="ETH"
    )
except BinanceAPIException as e:
    print(e)
else:
    print("Success")

## Transfer from MetaMask wallet to Binacne

In [ ]:
binance_address = client.get_deposit_address(coin='BUSD')["address"]
binance_address = Web3.toChecksumAddress(binance_address.lower())
binance_address

In [ ]:
quantity = 1827

contract = w3.eth.contract(address=busd, abi=abi)
decimals = contract.functions.decimals().call()
txn_amount = int(quantity * 10 ** decimals)

balance = contract.functions.balanceOf(address).call() * 10 ** -decimals 
print("Existing balance: {}".format(balance))


transaction = contract.functions.transfer(binance_address, txn_amount)

# has to include 'from: adress'
gas = transaction.estimateGas({'from': address})
print("Gas: {}".format(gas))

transaction = transaction.buildTransaction({
                   'chainId': 1,  # 1 for production chain
                   'gas':700000, 
                   'nonce': w3.eth.getTransactionCount(address)})
transaction

In [ ]:
signed_txn = w3.eth.account.signTransaction(transaction, '0x'+private_key)
txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
print("https://etherscan.io/tx/{}".format(str(txn_hash.hex())))

## Check Binance withdraw history

In [ ]:
withdraws = client.get_withdraw_history(coin='DAI')
withdraws